In [40]:
!pip install lightfm

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import the necessary libraries

In [42]:
import os
import random
import numpy as np
import pandas as pd
from scipy import sparse
import lightfm
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the dataset

In [43]:
df_books = pd.read_csv('/content/drive/MyDrive/Dataset/Books.csv', on_bad_lines='skip', encoding='latin-1')
df_users = pd.read_csv('/content/drive/MyDrive/Dataset/Users.csv', on_bad_lines='skip', encoding='latin-1')
df_ratings = pd.read_csv('/content/drive/MyDrive/Dataset/Ratings.csv', on_bad_lines='skip',
                          skiprows=lambda i: i>0 and random.random() > 0.50)

<ipython-input-43-861605dd2529>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('/content/drive/MyDrive/Dataset/Books.csv', on_bad_lines='skip', encoding='latin-1')


In [44]:
# what are the data types of all the columns that are present
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [45]:
df_books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [46]:
# Lets remane some wierd columns name
df_books.rename(columns={"Book-Title":'title',
                      'Book-Author':'author',
                     "Year-Of-Publication":'year',
                     "Publisher":"publisher",
                     "Image-URL-L":"image_url"},inplace=True)

In [47]:
df_books.head()

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [48]:
df_books.iloc[237]['image_url']

'http://images.amazon.com/images/P/0671027387.01.LZZZZZZZ.jpg'

In [49]:
# what are the data types of all the columns that are present
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [50]:
# Lets remane some wierd columns name
df_users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

In [51]:
df_users.tail()

,user_id,location,age
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN
278857,278858,"dublin, n/a, ireland",NaN


In [52]:
# what are the data types of all the columns that are present
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574544 entries, 0 to 574543
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      574544 non-null  int64 
 1   ISBN         574544 non-null  object
 2   Book-Rating  574544 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 13.2+ MB


In [53]:
# Lets remane some wierd columns name
df_ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [54]:
df_ratings.tail()

,user_id,ISBN,rating
574539,276704,080410526X,0
574540,276704,1563526298,9
574541,276706,0679447156,0
574542,276709,0515107662,10
574543,276721,0590442449,10


In [55]:
# Shape of the dataframe
print(df_books.shape, df_users.shape, df_ratings.shape, sep='\n')


(271360, 8)
(278858, 3)
(574544, 3)


In [56]:
df_ratings['user_id'].value_counts()

user_id
11676     6800
198711    3784
153662    3086
98391     2968
35859     2925
          ... 
133755       1
133684       1
133692       1
133695       1
133772       1
Name: count, Length: 71337, dtype: int64

#### Check for missing values


In [57]:
df_books.isnull().sum()

ISBN           0
title          0
author         2
year           0
publisher      2
Image-URL-S    0
Image-URL-M    0
image_url      3
dtype: int64

In [58]:
df_users.isnull().sum()

user_id          0
location         0
age         110762
dtype: int64

In [59]:
df_ratings.isnull().sum()

user_id    0
ISBN       0
rating     0
dtype: int64

In [60]:
# Check the duplicates values
print(df_books.duplicated().sum())
print(df_users.duplicated().sum())
print(df_ratings.duplicated().sum())

0
0
0


In [61]:
# Handling missing values
# replacing the null valuess with null string
df_books['publisher'].fillna('', inplace=True)
df_books['author'].fillna('', inplace=True)
df_books['image_url'].fillna('', inplace=True)

# replacing the null valuess with median value of age
df_users['age'].fillna(df_users['age'].median(), inplace=True)

In [62]:
print(df_books.isnull().sum())
print('===========')
print(df_users.isnull().sum())
print('===========')
df_ratings.isnull().sum()

ISBN           0
title          0
author         0
year           0
publisher      0
Image-URL-S    0
Image-URL-M    0
image_url      0
dtype: int64
user_id     0
location    0
age         0
dtype: int64


user_id    0
ISBN       0
rating     0
dtype: int64

In [63]:
df_ratings = df_ratings.groupby('ISBN').filter(lambda x: len(x)>=100)
df_ratings

,user_id,ISBN,rating
18,276755,0451166892,5
84,276832,0316781266,0
216,276925,0316666343,0
235,276925,0971880107,0
320,276939,0971880107,0
...,...,...,...
574405,276680,0375727345,8
574411,276680,0385335482,0
574440,276680,0679781587,9
574478,276683,0345337662,0


In [64]:
df_ratings =df_ratings.groupby('ISBN').filter(lambda x: len(x)>=100)
df_ratings

,user_id,ISBN,rating
18,276755,0451166892,5
84,276832,0316781266,0
216,276925,0316666343,0
235,276925,0971880107,0
320,276939,0971880107,0
...,...,...,...
574405,276680,0375727345,8
574411,276680,0385335482,0
574440,276680,0679781587,9
574478,276683,0345337662,0


In [65]:
df_ratings = df_ratings[df_ratings.groupby('user_id').ISBN.transform('nunique')>=10]
df_ratings

,user_id,ISBN,rating
751,277427,0060934417,0
774,277427,0312966091,0
777,277427,0316096199,0
779,277427,0316284955,0
781,277427,0316666343,0
...,...,...,...
573027,275970,0440212561,0
573029,275970,0440226430,0
573043,275970,044651652X,6
573112,275970,0553272535,0


In [66]:
def create_interaction_matrix(df, user_col, item_col, rating_col, norm=False, threshold=None):
    interactions = df.groupby([user_col, item_col])[rating_col] \
        .sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

# Create the interaction matrix
interactions = create_interaction_matrix(df=df_ratings, user_col='user_id', item_col='ISBN', rating_col='rating', norm=False)
interactions.head()


ISBN,0060392452,0060502258,0060915544,0060928336,0060930535,0060934417,0060938455,0060959037,0060976845,0060977493,...,0804114986,080411868X,0805063897,0842329129,0971880107,1400031354,1400034779,155874262X,1558743669,1573229326
user_id,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict



# Create user dictionaries
user_dict = create_user_dict(interactions=interactions)
print(user_dict)




{254: 0, 882: 1, 2766: 2, 3363: 3, 4017: 4, 5903: 5, 6242: 6, 6251: 7, 6543: 8, 6563: 9, 6575: 10, 7158: 11, 7283: 12, 7346: 13, 7915: 14, 8245: 15, 8253: 16, 8362: 17, 8681: 18, 9747: 19, 9856: 20, 10314: 21, 10447: 22, 11601: 23, 11676: 24, 12538: 25, 13273: 26, 13552: 27, 13582: 28, 14422: 29, 14521: 30, 14768: 31, 15408: 32, 15602: 33, 16106: 34, 16488: 35, 16634: 36, 16795: 37, 16916: 38, 17950: 39, 19664: 40, 20115: 41, 20180: 42, 21014: 43, 21404: 44, 21659: 45, 22625: 46, 23768: 47, 24921: 48, 25131: 49, 25981: 50, 26544: 51, 26593: 52, 28177: 53, 28204: 54, 28523: 55, 28634: 56, 29259: 57, 30276: 58, 30487: 59, 30533: 60, 30711: 61, 30716: 62, 30735: 63, 30972: 64, 31315: 65, 31556: 66, 31919: 67, 31987: 68, 32440: 69, 32773: 70, 33974: 71, 34588: 72, 35050: 73, 35836: 74, 35857: 75, 35859: 76, 36606: 77, 36609: 78, 36715: 79, 36836: 80, 37712: 81, 37950: 82, 38023: 83, 39281: 84, 39646: 85, 39773: 86, 40889: 87, 40943: 88, 41841: 89, 43246: 90, 43619: 91, 43687: 92, 43806: 93

In [68]:
def create_item_dict(df, id_col, name_col):
    item_dict = {}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i, id_col])] = df.loc[i, name_col]
    return item_dict

# Create user and item dictionaries
user_dict = create_user_dict(interactions=interactions)
#print(user_dict)

item_dict = create_item_dict(df=df_books, id_col='ISBN', name_col='title')

In [69]:

# Convert interaction matrix to sparse matrix
x = sparse.csr_matrix(interactions.values)

# Split into train and test sets
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=42)


In [70]:

def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30, n_jobs=4):
    model = LightFM(no_components=n_components, loss=loss, k=k)
    model.fit(interactions, epochs=epoch, num_threads=n_jobs)
    return model

# Train the model
model = runMF(interactions=train, n_components=30, loss='warp', k=15, epoch=30, n_jobs=4)


In [71]:

# Evaluate the model
train_auc = auc_score(model, train, num_threads=4).mean()
print('Train AUC: %s' % train_auc)

test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Test AUC: %s' % test_auc)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Train Precision: %.2f, Test Precision: %.2f' % (train_precision, test_precision))


Train AUC: 0.99935883
Test AUC: 0.54977745
Train Precision: 0.41, Test Precision: 0.01


In [72]:
def sample_recommendation_user(model, interactions, user_id, user_dict, item_dict, threshold=0, nrec_items=10, show=True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x, np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))

    known_items = list(pd.Series(interactions.loc[user_id, :][interactions.loc[user_id, :] > threshold].index).sort_values(ascending=False))

    # Ensure the known_items and scores are in the item_dict
    known_items = [item for item in known_items if item in item_dict]
    scores = [item for item in scores if item in item_dict]

    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))

    if show:
        print("Known Ratings:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter += 1

        print("\nRecommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter += 1
    return return_score_list



In [73]:
user_id = 62891
rec_list = sample_recommendation_user(model=model, interactions=interactions, user_id=user_id, user_dict=user_dict, item_dict=item_dict, threshold=0, nrec_items=10, show=True)


Known Ratings:
1- ANGELA'S ASHES
2- The English Patient
3- Hannibal
4- Red Dragon
5- Rising Sun
6- Congo
7- The Lovely Bones: A Novel

Recommended Items:
1- The Lovely Bones: A Novel
2- Red Dragon
3- Interview with the Vampire
4- The English Patient
5- Congo
6- Hannibal
7- Rising Sun
8- Along Came a Spider (Alex Cross Novels)
9- The Rainmaker
10- Tears of the Giraffe (No.1 Ladies Detective Agency)


In [74]:
user_id = 277427
rec_list = sample_recommendation_user(model=model, interactions=interactions, user_id=user_id, user_dict=user_dict, item_dict=item_dict, threshold=0, nrec_items=10, show=True)




Known Ratings:
1- The Partner
2- Me Talk Pretty One Day

Recommended Items:
1- Me Talk Pretty One Day
2- The Partner
3- We Were the Mulvaneys
4- The Vampire Lestat (Vampire Chronicles, Book II)
5- To Kill a Mockingbird
6- Bel Canto: A Novel
7- Dance upon the Air (Three Sisters Island Trilogy)
8- Confessions of a Shopaholic (Summer Display Opportunity)
9- The Cider House Rules
10- House of Sand and Fog


In [75]:
user_id = 4017
rec_list = sample_recommendation_user(model=model, interactions=interactions, user_id=user_id, user_dict=user_dict, item_dict=item_dict, threshold=0, nrec_items=10, show=True)

Known Ratings:
1- Angela's Ashes: A Memoir
2- SHIPPING NEWS
3- We Were the Mulvaneys
4- To Kill a Mockingbird
5- Into the Wild
6- Vinegar Hill (Oprah's Book Club (Paperback))
7- Songs in Ordinary Time (Oprah's Book Club (Paperback))
8- The Bean Trees

Recommended Items:
1- Vinegar Hill (Oprah's Book Club (Paperback))
2- Angela's Ashes: A Memoir
3- We Were the Mulvaneys
4- The Bean Trees
5- Into the Wild
6- Fall On Your Knees (Oprah #45)
7- Good in Bed
8- The Red Tent (Bestselling Backlist)
9- Bridget Jones's Diary
10- The Handmaid's Tale
